## 4.2 Loss Function 損失関数
### 4.2.1 Sum of Squared error
2乗和誤差
$$
    % 空白は表示に影響しない。コメントは"%"で始める
    % 下付き文字は"_a"、上付き文字は"^a"
    % 改行は"\\"を付ける
    %y_1 = ax^2 \\
    % 複数文字を1要素とする際は{...}で囲う
    % 空白は"\quad"
    %y_2 = ax^{10} \quad y_3 = ax \\
    %
    E = \frac{1}{2} \sum_{k} (y_k - t_k) ^2
$$

In [ ]:
# Import
import numpy as np
import matplotlib.pylab as plt


In [ ]:
# 4.2.1 Sum of Squared error

y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
t_dummy = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

def sum_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

print("Correct: ", sum_squared_error(np.array(y),np.array(t)))
print("Error  : " , sum_squared_error(np.array(y), np.array(t_dummy)))

### 4.2.2 Cross entropy error
交差エントロピー誤差

$$
    E = - \sum_{k} t_k \log (y_k) \\
$$

One-hotの場合は正解ラベルのみ'1'であるため実際には正解ラベルの対数(2)だけ計算すればいい

$$
    E = - \log {y_k}
$$




In [ ]:
print(-np.log(0.6))

x = np.arange(0.001, 1, 0.01)
plt.plot(x, np.log(x))
plt.xlim(0,1)
plt.ylim(-5 ,0)

plt.show()

In [ ]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

cross_entropy_error(np.array(y), np.array(t))

### 4.2.3 Mini Batch Learning

交差エントロピー誤差をバッチ適用する場合は、以下の式になる

$$
    E = - \frac{1}{N} \sum_n \sum_k t_{nk} \log(y_{nk})
$$



In [ ]:
## 4.2.3 Mini Batch Learning

import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape)

In [ ]:
## 4.2.4 Batch   cross entropy error

### for One-shot test result
def cross_entropy_error(y, t):
    if y.ndim ==1 :
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

### for label data
def cross_entropy_error2(y, t):
    if y.ndim ==1 :
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y[np.arange(batch_size), t] + 1e-7) / batch_size)

#事前確認
test = np.array([[1,2,3,4,5],[2,3,4,5,6]])
test2 = np.array([4,1])
print(test)
print("test dim = ",test.ndim)
print("test size= ",test.size)
batch = np.arange(test.shape[0]) #2行5列の　2行を0,1に分解　　[0 1]
test[batch, test2] #[0 1], [4, 1] 0行4列=5, 1行1列=3

##
train_size = x_train.shape[0] #60000
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)

x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]


## 4.3 Numerical differentiation 数値微分

### 4.3.1 Differentiation

ちなみに、普通の微分は以下。。。
$$
    \frac{df(x)}{dx} = \lim_{h \to 0} \frac{f(x+h)-f(x)}{h}
$$

In [ ]:
# 中心差分
def numerical_diff(f, x):
    h = 1e-4 # 0.0001
    return (f(x+h) - f(x-h)) / (2*h)

### 4.3.2 Example of Numerical differentiation

In [ ]:
def function_1(x):
    return 0.01*x**2 + 0.1*x

def tangent_line(f, x):
    slope = numerical_diff(f, x)
    intercept = f(x) - slope * x
    return lambda t: slope * t + intercept

import numpy as np
import matplotlib.pylab as plt

x = np.arange(0.0, 20.0, 0.1) # 0から20まで、0.1刻みのx配列
y = function_1(x)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.plot(x, y)

# x = 5 の接線
x_point = 5
tangent = tangent_line(function_1, x_point)
y_tangent = tangent(x)
plt.plot(x, y_tangent)

# x = 5 と function_1 の交わる点
plt.plot([x_point, x_point], [-1, function_1(5)], linestyle="--", color="gray")
plt.plot([0, x_point], [function_1(5), function_1(5)], linestyle="--", color="gray")
plt.xlim(0, 20)
plt.ylim(-1 , 6)

plt.show()


### 4.3.3 Partial Differentiation 偏微分

$$
    f{(x_0, x_1)} = x_0^2 + x_1^2
$$

In [ ]:
def function_2(x):
    return x[0]**2 + x[1]**2


# データ生成
x0 = np.linspace(-5, 5, 100)  # x0の範囲と分割数
x1 = np.linspace(-5, 5, 100)  # x1の範囲と分割数
x0, x1 = np.meshgrid(x0, x1)
y = function_2([x0, x1])

# 3Dプロット
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x0, x1, y, cmap='viridis')

# グラフにラベルを追加
ax.set_xlabel('X0')
ax.set_ylabel('X1')
ax.set_zlabel('function_2(X0, X1)')

# グラフを表示
plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 関数定義
def f(x):
    return x[0]**2 + x[1]**2

# データ生成
x0 = np.linspace(-5, 5, 100)  # x0の範囲と分割数
x1 = np.linspace(-5, 5, 100)  # x1の範囲と分割数
x0, x1 = np.meshgrid(x0, x1)
y = f([x0, x1])

# 3Dプロット
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x0, x1, y, cmap='viridis')

# グラフにラベルを追加
ax.set_xlabel('X0')
ax.set_ylabel('X1')
ax.set_zlabel('f(X0, X1)')

# グラフに点を追加
x0_point = 3
x1_point = 4
y_point = f([x0_point, x1_point])
ax.scatter(x0_point, x1_point, y_point, color='red', s=100, label='Point (3, 4)')
ax.quiver(x0_point, x1_point, y_point, df_dx0, df_dx1, 0.1, color='blue', label='Gradient')

# 偏微分の計算
df_dx0 = 2 * x0_point
df_dx1 = 2 * x1_point

# 偏微分の結果を表示
print("df/dx0 at (3, 4):", df_dx0)
print("df/dx1 at (3, 4):", df_dx1)

# グラフを表示
plt.show()

## 4.4 Gradient 勾配

勾配＝全ての変数の偏微分をベクトルとしてまとめたもの

勾配が示す方向は、各場所において関数の値を最も減らす方向

In [ ]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x) 
    
    for idx in range(x.size):
        tmp_val = x[idx]
        
        # f(x + h)の計算
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        # f(x-h)の計算
        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 値を元に戻す

    return grad

 
numerical_gradient(function_2, np.array([3.0, 4.0]))                

In [ ]:
def _numerical_gradient_no_batch(f, x):
    h = 1e-4  # 0.0001
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)  # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x)  # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val  # 値を元に戻す
        
    return grad


def numerical_gradient(f, X):
    if X.ndim == 1:
        return _numerical_gradient_no_batch(f, X)
    else:
        grad = np.zeros_like(X)
        print('grad shape= ', grad.shape)
        
        for idx, x in enumerate(X):
            grad[idx] = _numerical_gradient_no_batch(f, x)
        
        return grad

x0 = np.arange(-2, 2.5, 0.25)
x1 = np.arange(-2, 2.5, 0.25)
X, Y = np.meshgrid(x0, x1)

X = X.flatten()
Y = Y.flatten()

grad = numerical_gradient(function_2, np.array([X, Y]).T).T

plt.figure()
plt.quiver(X, Y, -grad[0], -grad[1],  angles="xy",color="#666666")
plt.xlim([-2, 2])
plt.ylim([-2, 2])
plt.xlabel('x0')
plt.ylabel('x1')
plt.grid()
plt.draw()
plt.show()

print(np.array([X, Y]).T.shape)
print(grad.shape)

### 4.4.1 Gradient method 勾配法

です。勾配法では、現在の場所から勾配方向に一定の距離だけ進みます。そして、移動した先でも同様に勾配を求め、また、その勾配方向へ進むというように、繰り返し勾配方向へ移動します。このように勾配方向へ進むことを繰り返すことで、関数の値を徐々に減らすのが勾配法（gradient method）です。



$$
    x_0 = x_0 - η \frac {\partial f}{\partial x_0} \\
    x_1 = x_1 - η \frac {\partial f}{\partial x_1} \
$$
η = Learning Rate

学習率を勾配にかけて元の値からひく


gradient descent_勾配降下法



In [ ]:
def gradient_descent(f, init_x, lr=0.1, step_num=20):
    x = init_x
    x_history = []
    
    for i in range(step_num):
        x_history.append(x.copy())
        
        grad = numerical_gradient(f, x)
        x -= lr * grad
        
        #plt.plot([x[0],])
        
    return x, np.array(x_history)

init_x = np.array([-3.0, 4.0])

x, x_history = gradient_descent(function_2, init_x)

# print(x_history)
plt.plot(x_history[:,0],x_history[:,1],'o')
plt.show()

### 4.4.2 Gradient for Neural Network

$$
    W = 
    \begin{pmatrix}
    w_{11} & w_{12} & w_{13} \\
    w_{21} & w_{22} & w_{32}
    \end{pmatrix} 
$$

$$
    \frac{\partial L}{\partial W} = 
    \begin{pmatrix}
    \frac{\partial L}{\partial w_{11}} & \frac {\partial L}{\partial w_{12}} & \frac {\partial L}{\partial w_{13}} \\
    \frac {\partial L}{\partial w_{21}} & \frac {\partial L}{\partial w_{22}} & \frac {\partial L}{\partial w_{23}}
    \end{pmatrix}     
$$

以下の例では、LのW11による偏微分の結果が0.3であった。
これは、w11をh増やした時に損失関数が0.3h増加することを意味する

In [ ]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # ガウス分布で初期化

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

net = simpleNet()
print(net.W)

x = np.array([0.6, 0.9])
t = np.array([0, 0, 1])

print(net.predict(x))
net.loss(x, t)

def f(W):
    return net.loss(x, t)

dw = numerical_gradient(f, net.W)
print('dw = ',dw)

## 　4.5 Implementation of Learning Algorithms

学習のステップのおさらい。学習は以下の手順で行われる。

1. ミニバッチ学習：訓練データからランダムに一部データを選び出し損失関数を値を計算
2. 勾配の算出：ミニバッチの喪失感すうを減らすために勾配を算出する
3. パラメータの更新：重みパラメータを勾配方向に微少量だけ更新する
4. #1~#3を繰り返す

上の方法でパラメータを更新する方法を確立的勾配降下法(stochastic gradient descent)という。

### 4.5.1 Class for a 2-layers Neural Network


In [ ]:
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size,
                 weight_init_std=0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y

    def loss(self, x, t):
        y = self.predict(x)
        print(y)

        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

#######
input_size = 784
hidden_size = 100
output_size = 10
net = TwoLayerNet(input_size, hidden_size, output_size)

x = np.random.rand(100, 784)
y = net.predict(x)
t = np.random.rand(100,10)
    

# print(net.params)

net.numerical_gradient(x, t)


###  4.5.2 Implementation of Mini-Batch Learning



In [ ]:
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)


train_loss_list = []

# ハイパーパラメータ
iters_num = 5
train_size = x_train.shape[0]
batch_size = 2
learning_rate = 0.1

network = TwoLayerNet(input_size=10, hidden_size=50, output_size=10)

for i in range(iters_num):
    # ミニバッチの取得
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 勾配の計算
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch) # 高速版!

    # パラメータの更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 学習経過の記録
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

#plt.plot(np.linspace(0, iters_num, iters_num -1), trains_loss_list[:])

### メモ

イテレーション10000を試しに回そうとしたら、2h強でギブ
100でトライアル18:49から1909でも終わらず　1920 でも終わらず　22でギブ

イテレーション5 1922〜
バッチ100が結構重い説
五分ほどで完了

-> バッチを10、イテレーションを５０で再確認　1929

・計算しっぱなしでおいておくとメモリ使用量が大きくなり、notebook開けなくなる。

In [ ]:
###### 
from PIL import Image
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

img = x_train[101]
label = t_train[101]
print(label)

print(img.shape)
img = img.reshape(28, 28)
print(img.shape)

img_show(img*256)

np.linspace(0,100,101)

### 4.5.3 Evaluate with Test Data

エポック（epoch）とは単位を表します。1エポックとは学習において訓練データをすべて使い切ったときの回数に対応します。たとえば、10,000個の訓練データに対して100個のミニバッチで学習する場合、確率的勾配降下法を100回繰り返したら、すべての訓練データを“見た”ことになります。この場合、100回＝1エポックとなります。

斎藤 康毅. ゼロから作る DeepLearning (p.205). Kindle 版. 

